# 用表格辨識器分析收據

![一個傀儡程式正持有一份收據](./images/receipt_analysis.jpg)

在電腦視覺的 [人工智慧 (AI)] 欄位中，光學字元辨識 (OCR) 一般用於讀取列印或手寫文件。通常，文字直接從文件中擷取到一種可以用於進一步處理或分析的格式中。

詳細的進階 OCR 案例是從表單 (例如購買訂單或發票) 中擷取資訊，並且提供對表單中欄位所代表含義的語意理解。**表格辨識器**服務是為這類 AI 問題特別設計的。

## 檢視收據

在此範例中，您可以使用表格辨識器的內建模型來分析收據。

按一下下方 **[執行儲存格]** (&#9655;) 按鈕 (儲存格左側) 以執行儲存格並參閱您將使用表格辨識器分析收據的範例。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline


# 載入並顯示收據影像
fig = plt.figure(figsize=(6, 6))
image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

## 建立表格辨識器資源

> **備註：**您要麼使用認知服務資源，要麼使用表格辨識器資源來存取表格辨識器服務。 

在您的 Azure 訂用帳戶中建立表格辨識器資源：

1.在其它瀏覽器索引標籤中，透過 https://portal.azure.com 開啟 Azure 入口網站，並用您的 Microsoft 帳戶登入。
2.選取 **[+ 建立資源]**，並搜尋*表格辨識器*。
3.在服務清單中，選取 **[表格辨識器]**。
4.在 **[表格辨識器]** 刀鋒視窗中，選取 **[建立]**。
5.在 **[建立]**刀鋒視窗中，輸入下列詳細資料並選取 **[建立]**
   - **名稱**：您的服務之唯一名稱
   - **訂用帳戶**：您的 Azure 訂用帳戶
   - **區域**：任一可用區域
   - **定價層**：F0
   - **資源群組**：您之前使用過的現有資源群組
   - **我確認已閱讀下方通知並理解通知內容**：已選取。
6.等待服務建立。
7.檢視您在 Azure 入口網站中新建立的表格辨識器服務並在 **[金鑰和端點]** 頁面，複製**金鑰 1** 和**端點**值，然後將他們貼上到下方程式碼儲存格中，取代 **YOUR_FORM_KEY** 和 **YOUR_FORM_ENDPOINT**。

In [ ]:
form_key = 'YOUR_FORM_KEY'
form_endpoint = 'YOUR_FORM_ENDPOINT'

print('Ready to use form recognizer at {} using key {}'.format(form_endpoint, form_key))

## 分析收據

現在您可以準備使用表格辨識器來分析收據。

In [ ]:
import os
from azure.ai.formrecognizer import FormRecognizerClient
from azure.core.credentials import AzureKeyCredential

# Create a client for the form recognizer service
form_recognizer_client = FormRecognizerClient(endpoint=form_endpoint, credential=AzureKeyCredential(form_key))

try:
    print("Analyzing receipt...")
    # Get the receipt image file
    image_path = os.path.join('data', 'form-receipt', 'receipt.jpg')

    # Submit the file data to form recognizer
    with open(image_path, "rb") as f:
        analyze_receipt = form_recognizer_client.begin_recognize_receipts(receipt=f)
    
    # Get the results
    receipt_data = analyze_receipt.result()

    # Print the extracted data for the first (and only) receipt
    receipt = receipt_data[0]
    receipt_type = receipt.fields.get("ReceiptType")
    if receipt_type:
        print("Receipt Type: {}".format(receipt_type.value))
    merchant_address = receipt.fields.get("MerchantAddress")
    if merchant_address:
        print("Merchant Address: {}".format(merchant_address.value))
    merchant_phone = receipt.fields.get("MerchantPhoneNumber")
    if merchant_phone:
        print("Merchant Phone: {}".format(merchant_phone.value))
    transaction_date = receipt.fields.get("TransactionDate")
    if transaction_date:
        print("Transaction Date: {}".format(transaction_date.value))
    print("Receipt items:")
    items = receipt.fields.get("Items")
    if items:
        for idx, item in enumerate(receipt.fields.get("Items").value):
            print("\tItem #{}".format(idx+1))
            item_name = item.value.get("Name")
            if item_name:
                print("\t - Name: {}".format(item_name.value))
            item_total_price = item.value.get("TotalPrice")
            if item_total_price:
                print("\t - Price: {}".format(item_total_price.value))
    subtotal = receipt.fields.get("Subtotal")
    if subtotal:
        print("Subtotal: {} ".format(subtotal.value))
    tax = receipt.fields.get("Tax")
    if tax:
        print("Tax: {}".format(tax.value))
    total = receipt.fields.get("Total")
    if total:
        print("Total: {}".format(total.value))

except Exception as ex:
    print('Error:', ex)


注意，表格辨識器能夠解譯表單中的資料，正確識別商家電子郵件地址和電話號碼、交易日期和時間，還有項目行、小計、稅金和總金額。

## 更多資訊

有關表格辨識器服務的更多資訊，請參閱[表格辨識器文件](https://docs.microsoft.com/zh-tw/azure/cognitive-services/form-recognizer/index)